In [1]:
import sys
import pyspark
from pyspark.sql import SparkSession

In [2]:
sys.path.append("/opt/workspace")
from load_env import env_parameter

# Load from .env
env_vars = env_parameter()

In [3]:
jdbc_url = env_vars["jdbc_url"]
db_user = env_vars["db_user"]
db_password = env_vars["db_password"]
spark_master = env_vars["spark_master"]
pg_jar = env_vars["pg_jar"]
event_log_dir = env_vars["event_log_dir"]
pg_driver = env_vars["pg_driver"]

In [4]:
# Start Spark session with PostgreSQL JDBC driver
spark = SparkSession.builder \
    .appName("ReadCRM") \
    .master(spark_master) \
    .config("spark.jars.packages", pg_jar) \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", event_log_dir) \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e846c818-19d4-4404-b078-44edb7553c4c;1.0
	confs: [default]
	found org.postgresql#postgresql;42.5.0 in central
	found org.checkerframework#checker-qual;3.5.0 in central
:: resolution report :: resolve 125ms :: artifacts dl 13ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.5.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------------------------

In [5]:
jdbc_properties = {
    "user": db_user,
    "password": db_password,
    "driver": pg_driver
}

In [9]:
# UDF 
def fetch_create_dataframe(spark, jdbc_url, table_name, jdbc_properties): 
    return spark.read.jdbc(url=jdbc_url, table=table_name, properties=jdbc_properties)

In [10]:
df_customer = fetch_create_dataframe(spark, jdbc_url, "customers", jdbc_properties)

In [11]:
df_customer.show()

+---+-----------------+-------------+-------------+--------+--------------------+-------------+--------------------+--------------------+
| id|             name|     industry|       region|   phone|             website|         tags|            metadata|          created_at|
+---+-----------------+-------------+-------------+--------+--------------------+-------------+--------------------+--------------------+
|  1|Northwind Traders|       Retail|North America|555-1001|https://northwind...|[active, key]|{"customer_tier":...|2025-06-02 17:19:...|
|  2|      Contoso Ltd|   Technology|       Europe|555-1002| https://contoso.com|   [prospect]|{"platform": "Azu...|2025-06-02 17:19:...|
|  3|  Adventure Works|Manufacturing|         Asia|555-1003|https://adventure...|        [new]|{"needs": "IoT in...|2025-06-02 17:19:...|
+---+-----------------+-------------+-------------+--------+--------------------+-------------+--------------------+--------------------+



In [12]:
df_users = fetch_create_dataframe(spark, jdbc_url, "users", jdbc_properties)

In [13]:
df_users.show()

+---+--------+--------------+--------------------+-------+--------------------+
| id|username|     full_name|               email|   role|          created_at|
+---+--------+--------------+--------------------+-------+--------------------+
|  1| jmartin|  Julia Martin|julia.martin@exam...|  sales|2025-06-02 17:19:...|
|  2| bwilson|    Ben Wilson|ben.wilson@exampl...|  sales|2025-06-02 17:19:...|
|  3|  lgreen|   Linda Green|linda.green@examp...|  sales|2025-06-02 17:19:...|
|  4| mkeller|Michael Keller|michael.keller@ex...|manager|2025-06-02 17:19:...|
+---+--------+--------------+--------------------+-------+--------------------+



In [14]:
df_contacts = fetch_create_dataframe(spark, jdbc_url, "contacts", jdbc_properties)

In [15]:
df_contacts.show()

+---+-----------+------------+--------------------+--------+---------------+--------------------+--------------------+
| id|customer_id|   full_name|               email|   phone|          title|               notes|          created_at|
+---+-----------+------------+--------------------+--------+---------------+--------------------+--------------------+
|  1|          1|Sarah Thomas|sarah.thomas@nort...|555-2001|            CTO|Decision-maker fo...|2025-06-02 17:19:...|
|  2|          2|Robert Allen|robert.allen@cont...|555-2002|Product Manager|Handles vendor ev...|2025-06-02 17:19:...|
|  3|          3| Emily Stone|emily.stone@adven...|555-2003|Operations Head|Focuses on proces...|2025-06-02 17:19:...|
+---+-----------+------------+--------------------+--------+---------------+--------------------+--------------------+



In [19]:
from pyspark.sql.functions  import col 
df_cust_contacts = df_contacts.alias("a").join(df_customer.alias("b"), col("a.customer_id") == col("b.id"), "inner")

In [20]:
df_cust_contacts.show()

[Stage 5:>                  (0 + 1) / 1][Stage 6:>                  (0 + 1) / 1]

+---+-----------+------------+--------------------+--------+---------------+--------------------+--------------------+---+-----------------+-------------+-------------+--------+--------------------+-------------+--------------------+--------------------+
| id|customer_id|   full_name|               email|   phone|          title|               notes|          created_at| id|             name|     industry|       region|   phone|             website|         tags|            metadata|          created_at|
+---+-----------+------------+--------------------+--------+---------------+--------------------+--------------------+---+-----------------+-------------+-------------+--------+--------------------+-------------+--------------------+--------------------+
|  1|          1|Sarah Thomas|sarah.thomas@nort...|555-2001|            CTO|Decision-maker fo...|2025-06-02 17:19:...|  1|Northwind Traders|       Retail|North America|555-1001|https://northwind...|[active, key]|{"customer_tier":...|20

In [21]:
spark.stop()